# Airline Arrivals #

* I will formulate this as a regression predicting excess lateness over 30 minutes.

* Flights late 30 minutes or less are coded as 0 minutes late.

* Dataset is from 1987 flights.

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

import warnings

from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
%matplotlib inline

In [2]:
pd.options.display.float_format = '{:.3f}'.format

In [3]:
airline = pd.read_csv("../data/1987.csv")
airline.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,1987,10,14,3,741.000,730,912.000,849,PS,1451,...,nan,nan,0,nan,0,nan,nan,nan,nan,nan
1,1987,10,15,4,729.000,730,903.000,849,PS,1451,...,nan,nan,0,nan,0,nan,nan,nan,nan,nan
2,1987,10,17,6,741.000,730,918.000,849,PS,1451,...,nan,nan,0,nan,0,nan,nan,nan,nan,nan
3,1987,10,18,7,729.000,730,847.000,849,PS,1451,...,nan,nan,0,nan,0,nan,nan,nan,nan,nan
4,1987,10,19,1,749.000,730,922.000,849,PS,1451,...,nan,nan,0,nan,0,nan,nan,nan,nan,nan


## Variable Descriptions ##


### Name	Description  ###

1	Year	1987-2008

2	Month	1-12

3	DayofMonth	1-31

4	DayOfWeek	1 (Monday) - 7 (Sunday)

5	DepTime	actual departure time (local, hhmm)

6	CRSDepTime	scheduled departure time (local, hhmm)

7	ArrTime	actual arrival time (local, hhmm)

8	CRSArrTime	scheduled arrival time (local, hhmm)

9	UniqueCarrier	unique carrier code

10	FlightNum	flight number

11	TailNum	plane tail number

12	ActualElapsedTime	in minutes

13	CRSElapsedTime	in minutes

14	AirTime	in minutes

15	ArrDelay	arrival delay, in minutes

16	DepDelay	departure delay, in minutes

17	Origin	origin IATA airport code

18	Dest	destination IATA airport code

19	Distance	in miles

20	TaxiIn	taxi in time, in minutes

21	TaxiOut	taxi out time in minutes

22	Cancelled	was the flight cancelled?

23	CancellationCode	reason for cancellation (A = carrier, B = weather, C = NAS, D = security)

24	Diverted	1 = yes, 0 = no

25	CarrierDelay	in minutes

26	WeatherDelay	in minutes

27	NASDelay	in minutes

28	SecurityDelay	in minutes

29	LateAircraftDelay	in minutes

In [4]:
airline.describe()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,TailNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,1311826.000,1311826.000,1311826.000,1311826.000,1292141.000,1311826.000,1288326.000,1311826.000,1311826.000,0.000,...,0.000,0.000,1311826.000,0.000,1311826.000,0.000,0.000,0.000,0.000,0.000
mean,1987.000,10.994,15.718,3.949,1369.276,1361.131,1493.152,1491.052,670.143,nan,...,nan,nan,0.015,nan,0.003,nan,nan,nan,nan,nan
std,0.000,0.823,8.836,1.986,478.718,471.970,498.750,486.800,514.551,nan,...,nan,nan,0.122,nan,0.054,nan,nan,nan,nan,nan
min,1987.000,10.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,nan,...,nan,nan,0.000,nan,0.000,nan,nan,nan,nan,nan
25%,1987.000,10.000,8.000,2.000,946.000,940.000,1117.000,1115.000,291.000,nan,...,nan,nan,0.000,nan,0.000,nan,nan,nan,nan,nan
50%,1987.000,11.000,16.000,4.000,1345.000,1338.000,1521.000,1519.000,560.000,nan,...,nan,nan,0.000,nan,0.000,nan,nan,nan,nan,nan
75%,1987.000,12.000,23.000,6.000,1750.000,1742.000,1918.000,1912.000,894.000,nan,...,nan,nan,0.000,nan,0.000,nan,nan,nan,nan,nan
max,1987.000,12.000,31.000,7.000,2400.000,2359.000,2400.000,2400.000,6282.000,nan,...,nan,nan,1.000,nan,1.000,nan,nan,nan,nan,nan


In [5]:
X = airline[['Month','DayOfWeek','CRSDepTime','CRSArrTime', 'UniqueCarrier','Origin',
             'Dest','Distance','DepDelay','Cancelled','Diverted']]
y = np.where(airline.ArrDelay > 30, airline.ArrDelay - 30, 0)            

In [6]:
X.head()

,Month,DayOfWeek,CRSDepTime,CRSArrTime,UniqueCarrier,Origin,Dest,Distance,DepDelay,Cancelled,Diverted
0,10,3,730,849,PS,SAN,SFO,447.000,11.000,0,0
1,10,4,730,849,PS,SAN,SFO,447.000,-1.000,0,0
2,10,6,730,849,PS,SAN,SFO,447.000,11.000,0,0
3,10,7,730,849,PS,SAN,SFO,447.000,-1.000,0,0
4,10,1,730,849,PS,SAN,SFO,447.000,19.000,0,0


In [7]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

X = MultiColumnLabelEncoder(columns = ['UniqueCarrier','Origin','Dest']).fit_transform(X)

In [8]:
y[:11]

array([0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0.])

In [9]:
X = X.fillna(X.mean())

In [10]:
X.describe()

,Month,DayOfWeek,CRSDepTime,CRSArrTime,UniqueCarrier,Origin,Dest,Distance,DepDelay,Cancelled,Diverted
count,1311826.000,1311826.000,1311826.000,1311826.000,1311826.000,1311826.000,1311826.000,1311826.000,1311826.000,1311826.000,1311826.000
mean,10.994,3.949,1361.131,1491.052,6.007,116.861,116.836,591.244,8.064,0.015,0.003
std,0.823,1.986,471.970,486.800,4.172,67.295,67.248,497.928,23.844,0.122,0.054
min,10.000,1.000,1.000,1.000,0.000,0.000,0.000,0.000,-1345.000,0.000,0.000
25%,10.000,2.000,940.000,1115.000,3.000,57.000,57.000,247.000,0.000,0.000,0.000
50%,11.000,4.000,1338.000,1519.000,6.000,122.000,122.000,417.000,0.000,0.000,0.000
75%,12.000,6.000,1742.000,1912.000,10.000,173.000,173.000,787.000,8.000,0.000,0.000
max,12.000,7.000,2359.000,2400.000,13.000,236.000,236.000,4983.000,1439.000,1.000,1.000


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [13]:
clr1 = ensemble.RandomForestRegressor()
cv1 = cross_val_score(clr1, X_train, y_train, cv=5)
print(cv1)
print("mean = {:.3}".format(cv1.mean()))

[0.88731795 0.86678607 0.89239609 0.86303431 0.86727508]
mean = 0.875


In [34]:
clr2 = ensemble.RandomForestRegressor(n_estimators=30, max_depth=3)
cv2 = cross_val_score(clr2, X_train, y_train, cv=5)
print(cv2)
print("mean = {:.3}".format(cv2.mean()))

[0.82964645 0.80443234 0.85167395 0.83678404 0.83996293]
mean = 0.832


In [15]:
clr3 = ensemble.RandomForestRegressor(n_estimators=30)
cv3 = cross_val_score(clr3, X_train, y_train, cv=5)
print(cv3)
print("mean = {:.3}".format(cv3.mean()))

[0.8934482  0.87201499 0.8992104  0.87741256 0.87358556]
mean = 0.883


In [13]:
# We'll make 100 iterations, use 2-deep trees, and set our loss function.
params = {'n_estimators': 100,
          'max_depth': 2,
          'loss': 'lad'}

# Initialize and fit the model.
gbr1 = ensemble.GradientBoostingRegressor(**params)
gb1 = cross_val_score(gbr1, X_train, y_train, cv=5)
print(gb1)
print("mean = {:.3}".format(gb1.mean()))


[0.70017519 0.67295257 0.72517479 0.72407359 0.72355387]
mean = 0.709


### Results ###

* clr3 wins.

* We will now run clr3 on the out of sample test dataset.


In [18]:
clr3.fit(X_train, y_train)
predict = clr3.predict(X_test)
r2 = clr3.score(X_test, y_test)
print("Final Out of Sample Test R2 is {:.3f}".format(r2))

Final Out of Sample Test R2 is 0.877


### Final Test Results ###

* Random Forest regression model with 30 trees works well out of sample.

* R2 result of 87.7% on test data.